# Lab 1: Welcome!

These labs will contain (1) review, and (2) any questions you ask that I can answer with examples. The idea behind this is that others likely have the same questions you do, and Jupyter notebooks provide a nice way to create minimal reproducible examples of common issues. Your questions don't have to be included if you don't want them to be.

## PEP8
https://www.python.org/dev/peps/pep-0008/

Eventually, these standards will be much like English grammar, where you will see something that looks wrong but not know why (like how there's an order to types of adjectives in English but we don't acknowledge it).

For now, adhere to the following:

* Four spaces for indentation (tabs are also fine as long as they're four spaces)
    * I recommend that you find and replace your tabs with four spaces before you submit
* No tabs AND spaces
* Max 79 characters / line
    * If you go over then use the reference [here](https://www.python.org/dev/peps/pep-0008/#indentation) on lining things up properly
    * You can break things up using nested parentheses or /
* Functions and class definitions should be preceded and followed by 2 blank lines
* Blank lines can be used to indicate logical separations or chunks
* Encoding should always be UTF-8 (and always will be if you use Python3)
* Put imports on different lines
    * Unless it's multiple from the same library like "from requests import get, post"
* Imports go at the top after docstrings
* Single vs. double quotes doesn't matter
    * Be consistent
* Put spaces before and after operators (<, =, ==, etc.)
* Use descriptive variables names
    * No x or whatever unless it's in a list comprehension or otherwise won't be used again
* Use snake case (underscores_between_words)
    * Except in class names, then use camelCase


In [2]:
"""
A code snippet that gets data from OpenWeatherMap
about the current weather at UChicago
but does it the WRONG WAY
"""

from requests import get
import requests, config
z = 60637
requestResponse=requests.get(f"http://api.openweathermap.org/data/2.5/weather?zip={z}&appid={config.open_weather_key}&units=imperial")
def prettyPrintWeather(requestResponse):
    requestResponse=requestResponse.json()
    i,j,k,l= requestResponse['weather'][0]['description'], requestResponse['main']['temp'], requestResponse['main']['feels_like'], requestResponse['name']
    print('The weather in ' + str(l) + ' is ' + str(i) + '. It is ' + str(j) +' degrees Farenheit, but feels like '+ str(k) +' degrees Farenheit.')
prettyPrintWeather(requestResponse)



The weather in Chicago is broken clouds. It is 63.52 degrees Farenheit, but feels like 47.07 degrees Farenheit.


In [3]:
"""
A code snippet that gets data from OpenWeatherMap
about the current weather at UChicago
but does it the RIGHT WAY
"""

import requests
import config


def pretty_print_weather(request_response):
    response_json = request_response.json()
    weather_description   = response_json['weather'][0]['description']
    actual_temperature    = response_json['main']['temp']
    perceived_temperature = response_json['main']['feels_like']
    city_name             = response_json['name']
    print(f'The weather in {city_name} is {weather_description}. '
          f'It is {actual_temperature} degrees Farenheit'
          f'but feels like {perceived_temperature} degrees Farenheit.')


zip_code = 60637
params = {'zip': zip_code,
          'appid': config.open_weather_key,
          'units': 'imperial'}

request_response = requests.get('http://api.openweathermap.org/data/2.5/weather', params = params)
    
pretty_print_weather(request_response)

The weather in Chicago is broken clouds . It is 63.52 degrees Farenheitbut feels like 47.07 degrees Farenheit.


## Pandas review

I'll just go through a MRE that uses Pandas to remind you of how things go. I'll grab some current weather data from major US cities and make it into a dataframe.

In [19]:
import pandas as pd

zip_codes = [10001, 94123, 60637, 90005, 98101, 80205, 33101, 77001, 20001]
weather_dict = {}
params = {'zip': 0,
          'appid': config.open_weather_key,
          'units': 'imperial'}

for zip_code in zip_codes:
    params['zip'] = zip_code
    response_json = requests.get('http://api.openweathermap.org/data/2.5/weather', params = params).json()
    weather_dict[zip_code] = [response_json['weather'][0]['description'],
                              response_json['main']['temp'],
                              response_json['main']['feels_like'],
                              response_json['name']]
weather_df = pd.DataFrame.from_dict(weather_dict, orient = 'index', columns = ['weather_desc', 'actual_temp',\
                                                                               'perceived_temp', 'city_name'])
weather_df.head(10)
    

,weather_desc,actual_temp,perceived_temp,city_name
10001,clear sky,69.40,59.72,New York
94123,clear sky,73.22,77.36,San Francisco
60637,broken clouds,62.80,47.48,Chicago
90005,clear sky,93.97,88.32,Los Angeles
98101,overcast clouds,66.47,66.85,Seattle
80205,few clouds,69.69,59.92,Denver
33101,broken clouds,87.53,92.57,Miami
77001,clear sky,83.53,79.32,Houston
20001,scattered clouds,69.89,59.34,Washington


Now I want to know some summary statistics! I'd like to have them in tabular and readable format. Note that you could also take the summary statistics by returning weather_df.describe(), but I'd like to make them myself.

In [24]:
mean_temp = weather_df['actual_temp'].mean().round(0)
sd = weather_df['actual_temp'].std().round(0)

weather_df.describe()


print(f'The mean temperature for these US cities is ~{mean_temp} and the standard deviation is ~{sd}.')

The mean temperature for these US cities is 75.0 and the standard deviation is 11.0.


,actual_temp,perceived_temp,temp_diff
count,9.000000,9.000000,9.000000
mean,75.166667,70.097778,7.193333
std,10.605483,15.099530,4.497166
min,62.800000,47.480000,0.380000
25%,69.400000,59.720000,4.210000
50%,69.890000,66.850000,5.650000
75%,83.530000,79.320000,9.770000
max,93.970000,92.570000,15.320000


I'm now interested in the difference between perceived temperature and actual temperature, so let's look into that.

In [26]:
weather_df['temp_diff'] = (weather_df['perceived_temp'] - weather_df['actual_temp']).apply(lambda x: abs(x))

weather_df = weather_df.sort_values(by = 'temp_diff', ascending = False)

weather_df.head(10)

,weather_desc,actual_temp,perceived_temp,city_name,temp_diff
60637,broken clouds,62.80,47.48,Chicago,15.32
20001,scattered clouds,69.89,59.34,Washington,10.55
80205,few clouds,69.69,59.92,Denver,9.77
10001,clear sky,69.40,59.72,New York,9.68
90005,clear sky,93.97,88.32,Los Angeles,5.65
33101,broken clouds,87.53,92.57,Miami,5.04
77001,clear sky,83.53,79.32,Houston,4.21
94123,clear sky,73.22,77.36,San Francisco,4.14
98101,overcast clouds,66.47,66.85,Seattle,0.38


In [ ]:
import seaborn as sns

